In [1]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip

--2025-03-31 14:32:21--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.1’

ml-100k.zip.1       100%[===================>]   4.70M  15.3MB/s    in 0.3s    

2025-03-31 14:32:22 (15.3 MB/s) - ‘ml-100k.zip.1’ saved [4924029/4924029]



In [2]:
!unzip ml-100k.zip -d ./ml-100k

Archive:  ml-100k.zip
replace ./ml-100k/ml-100k/allbut.pl? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./ml-100k/ml-100k/allbut.pl  
replace ./ml-100k/ml-100k/mku.sh? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./ml-100k/ml-100k/mku.sh  
replace ./ml-100k/ml-100k/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./ml-100k/ml-100k/README  
replace ./ml-100k/ml-100k/u.data? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./ml-100k/ml-100k/u.data  
replace ./ml-100k/ml-100k/u.genre? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./ml-100k/ml-100k/u.genre  
replace ./ml-100k/ml-100k/u.info? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./ml-100k/ml-100k/u.info  
replace ./ml-100k/ml-100k/u.item? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./ml-100k/ml-100k/u.item  
replace ./ml-100k/ml-100k/u.occupation? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./ml-100k/ml-100k/u.occupation  
replace ./ml-100k/ml-100k/u.user? [y]es, [n]o,

In [3]:
import pandas as pd

In [4]:
ratings = pd.read_csv("./ml-100k/ml-100k/u.data", sep="\t", names=["user_id", "item_id", "rating", "timestamp"])
print(ratings.head())

movies = pd.read_csv("./ml-100k/ml-100k/u.item", sep="|", encoding="latin-1", header=None,
                     names=["movie_id", "title", "release_date", "video_release_date", "IMDb_URL"] + [f"genre_{i}" for i in range(19)])
print(movies.head())

print("Number of users:", ratings["user_id"].nunique())
print("Number of items:", ratings["item_id"].nunique())

   user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596
   movie_id              title release_date  video_release_date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb_URL  genre_0  genre_1  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0        0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0        1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0        0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0      

In [5]:
user_item_matrix = ratings.pivot(index="user_id", columns="item_id", values="rating").fillna(0)
print(user_item_matrix.head())

item_id  1     2     3     4     5     6     7     8     9     10    ...  \
user_id                                                              ...   
1         5.0   3.0   4.0   3.0   3.0   5.0   4.0   1.0   5.0   3.0  ...   
2         4.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   2.0  ...   
3         0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
4         0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
5         4.0   3.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   

item_id  1673  1674  1675  1676  1677  1678  1679  1680  1681  1682  
user_id                                                              
1         0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2         0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3         0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4         0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
5         0.0   0.0   0.0   0.0   0.0   0.0   0

In [6]:
!pip install numpy==1.26.4
!pip install --no-cache-dir scikit-surprise

!pip install scikit-surprise

In [7]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

In [8]:
reader = Reader(rating_scale=(1, 5))
data_surprise = Dataset.load_from_df(ratings[["user_id", "item_id", "rating"]], reader)

svd = SVD(n_epochs=20, lr_all=0.005, reg_all=0.02)
cross_val_results = cross_validate(svd, data_surprise, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9375  0.9317  0.9366  0.9280  0.9428  0.9353  0.0051  
MAE (testset)     0.7362  0.7349  0.7396  0.7338  0.7437  0.7376  0.0036  
Fit time          2.78    3.13    4.72    3.68    1.45    3.15    1.08    
Test time         0.26    0.47    0.62    0.19    0.10    0.33    0.19    


In [9]:
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 44.1 MB/s eta 0:00:00


In [10]:
import numpy as np
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

In [11]:
sparse_user_item = csr_matrix(user_item_matrix.values)

als_model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)
als_model.fit(sparse_user_item.T)

user_id = 1
user_items = sparse_user_item[user_id-1]
recommendations = als_model.recommend(user_id-1, user_items)
print("Recommendations (item indices, scores):", recommendations)

/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
/usr/local/lib/python3.11/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.010445594787597656 seconds
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

Recommendations (item indices, scores): (array([621, 891, 653, 707, 619, 636, 867, 837, 785, 884], dtype=int32), array([1.428712 , 1.3402815, 1.258325 , 1.251547 , 1.2448912, 1.2287047,
       1.2014992, 1.1968485, 1.1950357, 1.1927984], dtype=float32))
